<a href="https://colab.research.google.com/github/Curiousss/EVA/blob/master/Assignment20/Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
TRAIN_BATCH_SIZE = 32


In [0]:
split = (80, 10, 10)
splits = tfds.Split.TRAIN.subsplit(weighted=split)
(train, valid, test), info = tfds.load('cifar100', split=list(splits), with_info=True, as_supervised=True)

In [0]:
def pre_process_image(image, label):
  image = tf.cast(image, tf.float32)
  image = image / 255.0
  #image = tf.image.resize(image, (64, 64))
  return image, label

In [0]:
train = train.map(pre_process_image).shuffle(1000).repeat().batch(TRAIN_BATCH_SIZE)
valid = valid.map(pre_process_image).repeat().batch(1000)

In [0]:
IMG_SHAPE = (32, 32, 3)
res_net = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
res_net.trainable = False

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
res_net.summary()
#for layer in res_net.layers[:-7]:
#  layer.trainable = False


In [0]:
res_net_out = res_net.get_layer('conv5_block1_out').output
#res_net.layers[-2].outbound_nodes = []
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(res_net_out)
output_layer = tf.keras.layers.Dense(100, activation='softmax')(global_average_layer)
tl_model = tf.keras.Model(res_net.input, output_layer)


In [0]:
tl_model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01, clipvalue=0.5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
#callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./log/transer_learning_model', update_freq='batch')]
tl_model.fit(train, steps_per_epoch = 23262//TRAIN_BATCH_SIZE, epochs=5, 
             validation_data=valid, validation_steps=10)#, callbacks=callbacks)


InternalError: ignored